In [ ]:
#@title codes to mount your google drive folder
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab_Notebooks/SMU_MITB_NLP/group_project/notebooks/

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/SMU_MITB_NLP/group_project/notebooks


In [ ]:
pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 46.9 MB/s eta 0:00:00


In [ ]:
import av
import numpy as np
import torch
from transformers import AutoImageProcessor, AutoTokenizer, VisionEncoderDecoderModel

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# load pretrained processor, tokenizer, and model
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = VisionEncoderDecoderModel.from_pretrained("Neleac/timesformer-gpt2-video-captioning").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/41.2k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
#check device if its gpu or not
device

'cuda'

In [ ]:
video_path = "./data/sample_game_phi_vs_mia.mp4"
container = av.open(video_path)
print(container)

<av.InputContainer './data/sample_game_phi_vs_mia.mp4'>


In [ ]:
# extract evenly spaced frames from video
seg_len = container.streams.video[0].frames
clip_len = model.config.encoder.num_frames
indices = set(np.linspace(0, seg_len, num=clip_len, endpoint=False).astype(np.int64))
frames = []
container.seek(0)
for i, frame in enumerate(container.decode(video=0)):
    if i in indices:
        frames.append(frame.to_ndarray(format="rgb24"))

# generate caption
gen_kwargs = {
    "min_length": 10,
    "max_length": 20,
    "num_beams": 8,
}
pixel_values = image_processor(frames, return_tensors="pt").pixel_values.to(device)
tokens = model.generate(pixel_values, **gen_kwargs)
caption = tokenizer.batch_decode(tokens, skip_special_tokens=True)[0]
print(caption)

/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)


A group of men are playing a game of basketball on a court.


In [ ]:
#create a function that gets the video path, and then generates the caption
#video_path = "./data/sample_game_phi_vs_mia.mp4"
container = av.open(video_path)
print(container)
gen_kwargs = {
        "min_length": 10,
        "max_length": 30,
        "num_beams": 8,
    }

def get_caption(video_path):
  container = av.open(video_path)
  print(container)
  gen_kwargs = {
          "min_length": 10,
          "max_length": 30,
          "num_beams": 8,
      }

  seg_len = container.streams.video[0].frames
  clip_len = model.config.encoder.num_frames

  indices = set(np.linspace(0, seg_len, num=clip_len, endpoint=False).astype(np.int64))
  frames = []
  container.seek(0)

  for i, frame in enumerate(container.decode(video=0)):
      if i in indices:
          frames.append(frame.to_ndarray(format="rgb24"))


  pixel_values = image_processor(frames, return_tensors="pt").pixel_values.to(device)
  print(pixel_values.shape)
  tokens = model.generate(pixel_values, **gen_kwargs)
  caption = tokenizer.batch_decode(tokens, skip_special_tokens=True)[0]

  return caption


<av.InputContainer './data/sample_game_phi_vs_mia.mp4'>


In [ ]:
get_caption(video_path)

'A group of men are playing a game of basketball on a court.'

# 2.0 Pipeline for Inference

the changes to make are:

(1) : temp_csv_paths

(2) : comment off [:-4]

In [ ]:
#@title get all the video paths first - containing dal

import os
import glob

video_paths = glob.glob("./data/raw/NSVA_Data/NSVA_Video/*")
print(len(video_paths))
video_paths[:5]

22


['./data/raw/NSVA_Data/NSVA_Video/game-0021800013-dal-vs-phx-video-10981.mp4',
 './data/raw/NSVA_Data/NSVA_Video/game-0021800013-dal-vs-phx-video-10978.mp4',
 './data/raw/NSVA_Data/NSVA_Video/game-0021800013-dal-vs-phx-video-10980.mp4',
 './data/raw/NSVA_Data/NSVA_Video/game-0021800013-dal-vs-phx-video-10979.mp4',
 './data/raw/NSVA_Data/NSVA_Video/game-0021800013-dal-vs-phx-video-10977.mp4']

In [ ]:
#@title get all the csv paths first - containing dal

import os
import glob

team_name = "dal"
csv_paths = glob.glob("./data/raw/NSVA_Data/NSVA_Data/*")
csv_paths = [path for path in csv_paths if team_name in path]
print(len(csv_paths))
csv_paths[:5]

29


['./data/raw/NSVA_Data/NSVA_Data/0021800210-uta-vs-dal.csv',
 './data/raw/NSVA_Data/NSVA_Data/0021800160-dal-vs-uta.csv',
 './data/raw/NSVA_Data/NSVA_Data/0021800109-dal-vs-lal.csv',
 './data/raw/NSVA_Data/NSVA_Data/0021800094-dal-vs-sas.csv',
 './data/raw/NSVA_Data/NSVA_Data/0021800084-uta-vs-dal.csv']

In [ ]:
import pandas as pd

#assume we pick one of the csv file, for now we hardcode it
ref_csv_path = "./data/raw/NSVA_Data/NSVA_Data/0021800013-dal-vs-phx.csv"
print(ref_csv_path in csv_paths)

temp_csv_paths = [ref_csv_path]
#temp_csv_paths = csv_paths

for csv_path in temp_csv_paths:
  #comment the index later
  df = pd.read_csv(ref_csv_path)[:4]
  # get every row, and check the video id
  game_name = csv_path.split("/")[-1].split(".")[0]
  for index, row in df.iterrows():
    video_id = row['video_id']
    # Check if there are multiple video IDs separated by </>
    if '</>' in video_id:
      # Split the string and access each video ID
      video_ids = video_id.split(' </> ')
      for video_id in video_ids:
        # Do something with the video ID
        print(2,game_name,video_id)
        mp4_path = game_name + '_' + video_id + ".mp4"
        print("./data/raw/NSVA_Data/NSVA_Video/" + mp4_path)
        caption = get_caption("./data/raw/NSVA_Data/NSVA_Video/" + mp4_path)
        print(caption)


    else:
      # There is only one video ID
      print(1,game_name,video_id)
      mp4_path = game_name + '_' + video_id + ".mp4"
      print("./data/raw/NSVA_Data/NSVA_Video/" + mp4_path)
      caption = get_caption("./data/raw/NSVA_Data/NSVA_Video/" + mp4_path)
      print(caption)
    print('\n')

True
1 0021800013-dal-vs-phx 10977
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10977.mp4
<av.InputContainer './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10977.mp4'>
torch.Size([1, 8, 3, 224, 224])
A basketball game is being played in front of a crowd.


2 0021800013-dal-vs-phx 10978
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10978.mp4
<av.InputContainer './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10978.mp4'>
torch.Size([1, 8, 3, 224, 224])
Two teams are playing a game of basketball in front of a crowd.
2 0021800013-dal-vs-phx 10979
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10979.mp4
<av.InputContainer './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10979.mp4'>
torch.Size([1, 8, 3, 224, 224])
Two teams are playing a game of basketball in front of a crowd.


2 0021800013-dal-vs-phx 10980
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10980.mp4
<av.InputContainer './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_1098